# Determining Optimal Infielder Positioning for Relaying Outfielders' Throws to Home Plate

Let's consider a common baseball play: with a runner on base, a ball is hit to an outfielder and the runner attempts to score. To prevent the runner from scoring, the outfielder can either throw the ball to home plate directly or throw the ball to an infielder (the so-called "cutoff man") who will then throw to home plate. As the ball (a projectile) travels through the air (a fluid), it is subjected to two distinct forces: the gravitational force (which acts downwards) and the resistive force, or drag, of the fluid. These forces tend to slow the ball as it travels towards home plate, so one advantage of using a cutoff man is to throw the ball a second time (thereby increasing its velocity) midway through its journey. However, there is an additional time cost incurred by this relay, as the cutoff man must catch, turn, and throw in a non-negligible time interval. So, the fielder(s) must decide which option is faster. The time taken for the ball to arrive at home plate is dependent on the initial distance between the outfielder and home plate, the outfielder's arm strength, and the viscosity of the fluid (air). If a cutoff man is involved, his position between the outfielder and home plate, his arm strength, and the time required to catch and release the ball must also be considered.

Here is an example of a spectacular relay throw:

<br>
<img src="ezgif.com-optimize.gif" alt="cruzcontrol" width="1400">
<br>

Here is another, equally effective relay "throw":

<br>
<img src="https://i.makeagif.com/media/9-20-2014/04r9Hx.gif" alt="re2pect" width="1400">
<br>

Baseball is weird sometimes. But that's what makes it fun!

## The Physics of a Throw

As mentioned previously, a thrown baseball is a two-dimensional projectile with mass $m$ and diameter $D$ that experiences both a gravitational force $\mathbf{w}=-mg \, \mathbf{\hat{j}}$ and a drag due to air resistance. (There may also be magnus forces due to the spin of the ball and seam-shifted wake effects, but these are far less significant and extremely difficult to model.) The drag is a function of the speed of the ball and directly opposes its movement; that is,

$$\mathbf{f}=-f(v) \, \mathbf{\hat{v}}\text{,}$$

where $\mathbf{\hat{v}} = \mathbf{v} / \left| \mathbf{v} \right|$ denotes the unit vector in the direction of the velocity $\mathbf{v}$. The magnitude of the drag $f$ can be written as the sum of a linear term $f_\text{lin}=bv$ and a quadratic term $f_\text{quad}=cv^2$.

Each of these terms has a physical significance. $f_\text{lin}$ arises from the viscous drag of the medium (the frictional forces between the fluid and the sides of the projectile) and is generally proportional to the viscosity of the medium and the \emph{linear} size of the projectile. $f_\text{quad}$ is a consequence of the projectile's having to accelerate the mass of fluid directly in front of it at any given instance. (That is, there is a non-viscous, nonlinear force exerted by the fluid that the projectile is continually having to "push out of the way".) For a spherical projectile, the linear and quadratic terms take the form

$$f(v)=f_\text{lin} + f_\text{quad} = \beta D v + \gamma D^2 v^2 \text{.}$$

Now, oftentimes one of these term is negligible compared to the other. Evaluating the ratio $f_\text{quad} / f_\text{lin}$ for a projectile with a diameter $d=7.5 \, \text{cm}$ ($2.9 \, \text{in}$) traveling at $v=40 \, \text{m} / \text{s}$ ($89 \, \text{mi} / \text{h}$) through air at standard temperature and pressure (STP) yields

$$\frac{f_\text{quad}}{f_\text{lin}} = \frac{\gamma D^2 v^2}{\beta D v} = \frac{\gamma}{\beta} D v = \frac{0.25 \, \text{N} \cdot \text{s}^2 / \text{m}^4}{1.6 \times 10^{-4} \, \text{N} \cdot \text{s} / \text{m}^2} \! \left(7.5 \times 10^{-2} \, \text{m} \right) \! \left(40 \, \text{m} / \text{s} \right) = 4.6 \times 10^3 \text{,}$$

meaning that the quadratic term dominates the linear term in this case and the drag can safely be modeled as

$$\mathbf{f} = -\gamma D^2 v^2 \, \mathbf{\hat{v}} \text{.}$$

The cell below generates an interactive plot demonstrating the interaction between projectile diameter and relative linear/quadratic drag strength. The horizontal axis shows speeds between $v=0 \text{m} / \text{s}$ and $v=40 \text{m} / \text{s}$. As the projectile diameter is increased, $f_\text{quad}$ increases much faster than $f_\text{lin}$ (since the former is proportional to $D^2$ and the latter to $D$). From this graphic, we see that the linear term is significant only for *very* small projectiles at relatively low speeds. For projectiles larger than $1 mm$ in diameter, the drag is due almost entirely to the inertia (rather than the viscosity) of the fluid.

In [3]:
from math import floor, log10

import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
import numpy as np

gamma = 2.5e-1
beta = 1.6e-4

# Create figure
fig = make_subplots(specs=[[{'secondary_y': True}]])

# Add traces, one for each projectile diameter (1 mm through 1 m)
vs = np.arange(0.1, 40.1, 0.1)
Ds = np.logspace(-6.0, -4.0, num=100)
for D in Ds:
    f_lins = beta * D * vs
    f_quads = gamma * (D * vs) ** 2
    for key, value in {r'$f_\text{lin}$': {'data': f_lins, 'color': 'red', 'secondary_y': False},
                       r'$f_\text{quad}$': {'data': f_quads, 'color': 'blue', 'secondary_y': False},
                       r'$f_\text{quad}/f_\text{lin}$': {'data': f_quads / f_lins, 'color': 'purple', 'secondary_y': True}}.items():
        fig.add_trace(
            go.Scatter(
                mode='lines',
                visible=False,
                line=dict(color=value['color'], width=3),
                name=key,
                x=vs,
                y=value['data']
            ),
            secondary_y=value['secondary_y']
        )
    
# Set the default to be 7.5 cm
for i in range(3):
    fig.data[0 * 3 + i].visible = True

# Create and add slider
steps = []
for i in range(len(fig.data) // 3):
    step = dict(
        method='update',
        args=[dict(visible=[False] * len(fig.data)),
              dict(title=f'Ratio of linear to quadratic drag <i>vs</i>. projectile velocity for a projectile with diameter {round(Ds[i] / 10 ** floor(log10(Ds[i])), 1)} × 10<sup>{floor(log10(Ds[i]))}</sup> m')],
        label=f'{round(Ds[i] / 10 ** floor(log10(Ds[i])), 1)}×10<sup>{floor(log10(Ds[i]))}</sup> m'
    )
    for j in range(3):
        step['args'][0]['visible'][3 * i + j] = True
    steps.append(step)

sliders = [dict(
    active=0,
    currentvalue=dict(prefix='D = ', font=dict(size=12)),
    pad={'t': 50},
    steps=steps,
    font=dict(size=8)
)]

fig.update_layout(
    title=f'Ratio of linear to quadratic drag <i>vs</i>. projectile velocity for a projectile with diameter {round(Ds[0] / 10 ** floor(log10(Ds[0])), 1)} × 10<sup>{floor(log10(Ds[0]))}</sup> m',
    sliders=sliders,
    legend=dict(
        xanchor='left',
        x=0.97,
        yanchor='top',
        y=1.0,
        entrywidthmode='fraction',
        entrywidth=0.99,
        itemwidth=30,
        font=dict(size=16)
    ),
    # legend=dict(yanchor='bottom', y=1.02, xanchor='left', x=0, font=dict(size=16), itemwidth=30, entrywidth=100, entrywidthmode='fraction'),
    hovermode=False
)
fig.update_xaxes(title=dict(text='<b>Velocity (m/s)</b>'), ticks='outside')
fig.update_yaxes(secondary_y=False, minallowed=0.0, title=dict(text='<b>Force (N)</b>'), exponentformat='power', ticks='outside')
fig.update_yaxes(secondary_y=True, title=dict(text='<b>Ratio</b>'), range=[0.0, 2.0], ticks='outside')

pio.renderers.default = 'browser'
fig.show()

Newton's second law for this system is

$$m \mathbf{a} = m \frac{d \mathbf{v}}{dt} = \mathbf{F}_\text{net} = m \mathbf{g} + \mathbf{f} = m \mathbf{g} - f(v) \, \mathbf{\hat{v}} \approx m \mathbf{g} - \gamma D^2 v^2 \mathbf{\hat{v}} = m \mathbf{g} - \gamma D^2 v \mathbf{v} = m \mathbf{g} - \gamma D^2 \sqrt{v_x^2 + v_y^2} \, \mathbf{v} \text{,}$$

or, in component form

$$\begin{align*} 
m \frac{d v_x}{dt} &= - \gamma D^2 \sqrt{v_x^2 + v_y^2} \, v_x \\ 
m \frac{d v_y}{dt} &= mg - \gamma \sqrt{v_x^2 + v_y^2} \, v_y \text{.}
\end{align*}$$

This pair of differential equations is coupled (since each equation involves both of the variables $v_x$ and $v_y$), making them impossible to solve in terms of elementary functions. As a first approximation, we can ignore the thrown baseball's vertical movement and view it as a one-dimensional projectile moving in the horizontal direction. This simplifying assumption is reasonable, since $v_x \gg v_y$ (so $\sqrt{v_x^2 + v_y^2} \approx v_x$) for the majority of a thrown ball's journey. Case in point:

<br>
<img src="frozenrope.gif" alt="re2pect" width="1400">
<br>

Not a lot of arc on that thing. So, the relevant differential equation is

$$m \frac{d v_x}{dt} = - \gamma D^2 v_x^2  \text{,}$$

which is solved as

$$\int \frac{d v_x}{v_x^2} = - \frac{1}{v_x} = -\frac{\gamma D^2}{m} \int dt = -\frac{\gamma D^2}{m} t + C \rightarrow v_x(t)=\frac{1}{(\gamma D^2 / m) t + C} \text{.}$$

Setting $t=0$ shows that $v_x(t)=v_0=1/C$, and taking $\tau = m / \left( \gamma D^2 v_0 \right)$ simplifies this equation to

$$v_x(t) = \frac{v_0}{1 + t / \tau} \text{.}$$